In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pandas as pd

import sys
sys.path.append("../../../src/")
sys.path.append("../../")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

## Hyperparameter

In [16]:
excel_path = "ALL.xlsx"
excel = pd.read_excel("ALL.xlsx", dtype={"exp": str, "selected_layer": int,
                                   "selected_filter": int}).values

In [32]:
for data in excel:
    exp, selected_layer, selected_filter = data[0], data[1], data[2]
    print(exp, selected_layer, selected_filter)

x052490 1 16
x7001 1 47
x7004 3 28
x060490 3 20
x7005 6 19
x7006 8 99
x70071 11 75
x70081 13 91
x7010 15 173
x7010 18 49
x060493 18 101
x7011 20 398
x7012 22 485
x7013 25 1


In [3]:
# exp = "052490"
# selected_layer = 1
# selected_filter = 16

# exp = "7001"
# selected_layer = 1
# selected_filter = 47

# exp = "7006"
# selected_layer = 8
# selected_filter = 99

# exp = "7013"
# selected_layer = 25
# selected_filter = 1

In [4]:
# selected_layer = 6
# selected_filter = 16

# class_index = 968
# exp = "052190"
# epoch = "46448"

# class_index = 878
# exp = "052191"
# epoch = "45632"

# class_index = 90
# exp = "052192"
# epoch = "50702"
# 
# class_index = 14
# exp = "052193"
# epoch = "46907"

# class_index = 950
# exp = "033104"
# epoch = "99900"

# exp = "040104"
# epoch = "99900"



# exp = "70071"
# exp = "9408"
# exp = "9409"
# selected_layer = 11
# selected_filter = 75

# exp = "70081"
# epoch = "149999"
# selected_layer = 13
# selected_filter = 91

# exp = "7009"
# epoch = "149999"
# selected_layer = 15
# selected_filter = 173

# exp = "7010"
# epoch = "149999"
# selected_layer = 18
# selected_filter = 49

# exp = "7011"
# epoch = "75129"
# selected_layer = 20
# selected_filter = 398

# exp = "7012"
# epoch = "79405"
# selected_layer = 22
# selected_filter = 485

# exp = "7004"
# exp = "70041"
# exp = "70042"
# epoch = "127203"
# selected_layer = 3
# selected_filter = 28

# exp = "7006"
# epoch = "149999"
# selected_layer = 8
# selected_filter = 99

# exp = "7000"
# epoch = "46785"
# selected_layer = 1
# selected_filter = 16


# exp = "060490"
# epoch = "116566"
# selected_layer = 3
# selected_filter = 20

# exp = "060491"
# epoch = "116566"
# selected_layer = 3
# selected_filter = 20
# 
# exp = "060492"
# epoch = "116566"
# selected_layer = 3
# selected_filter = 20
# 
# exp = "060493"
# epoch = "116566"
# selected_layer = 3
# selected_filter = 20

# exp = "060493"
# epoch = "61521"
# selected_layer = 18
# selected_filter = 101

# exp = "060301"
# exp = "052195"
# exp = "052196"
# exp = "052197"
# exp = "052198"
# selected_layer = 3
# selected_filter = 20

# exp = "060302"
# exp = "9400"
# exp = "9401"
# exp = "9402"
# exp = "9403"
# selected_layer = 6
# selected_filter = 19

# exp = "060304"
# selected_layer = 1
# selected_filter = 6

# exp = "060305"
# selected_layer = 1
# selected_filter = 6

# exp = "70002"
# exp = "70001"
# selected_layer = 1
# selected_filter = 16

# exp = "70011"
# exp = "70012"
# exp = "052201"
# exp = "052202"
# exp = "052203"
# exp = "052204"
# exp = "052190"
# selected_layer = 1
# selected_filter = 47

class_index = 950
# class_index = 968
# class_index = 878
# class_index = 90
# class_index = 14
num_class = 30

In [3]:
backbone = "vgg16"
resume = "037-0"
# color_map = "nipy_spectral"
color_map = "jet"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load modal from dict

In [5]:
# model
net = model.Network(backbone=backbone, num_classes=1000, 
                    selected_layer=27)
net.to(device)

# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt, strict=False) 

Only Keep 27th layers before.
Resume from model from exp: 037 at epoch 0


_IncompatibleKeys(missing_keys=[], unexpected_keys=['model.fc.0.weight', 'model.fc.0.bias', 'model.fc.3.weight', 'model.fc.3.bias', 'model.fc.6.weight', 'model.fc.6.bias', 'model.features.28.weight', 'model.features.28.bias'])

In [12]:
torch.mean(net.model.features[0].bias)

tensor(-0.0650, device='cuda:0', grad_fn=<MeanBackward0>)

In [7]:
summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## Load Original Data 

In [8]:
# Load data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
reverse_mean = [-0.485, -0.456, -0.406]
reverse_std = [1/0.229, 1/0.224, 1/0.225]
train_transform = None
train_transform = transforms.Compose([                                                                                                                                
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
    transforms.ToTensor(),                                                                                                                                            
    #transforms.Normalize(mean, std)                                                                                                                                   
       ])         
trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                             is_train=True, transform=train_transform)

trainset.set_data([class_index], num_class)
imgs_path = []                                                                                                                                                            
images = []
labels = []
for img, label, img_path in trainset:                                                                                                                                     
    images.append(img.unsqueeze(0))                                                                                                                                       
    labels.append(label)                                                                                                                                                  
    imgs_path.append(img_path)  

Len of new dataset is :30


## Create some need path

In [9]:
save_dir = "../../../saved/pack/"
################### Hyper-Parameter #######################
# exp = resume_exp
# epoch = resume_epoch
##########################################################
# ab_path = os.path.join(save_dir, exp, epoch)
ab_path = os.path.join(save_dir, exp)

In [10]:
index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
index2image

{0: 'n07747607_10061',
 1: 'n07747607_10068',
 2: 'n07747607_10110',
 3: 'n07747607_10134',
 4: 'n07747607_10162',
 5: 'n07747607_10166',
 6: 'n07747607_10178',
 7: 'n07747607_10179',
 8: 'n07747607_10184',
 9: 'n07747607_10192',
 10: 'n07747607_10204',
 11: 'n07747607_10205',
 12: 'n07747607_1021',
 13: 'n07747607_10213',
 14: 'n07747607_10217',
 15: 'n07747607_10232',
 16: 'n07747607_10258',
 17: 'n07747607_10290',
 18: 'n07747607_10305',
 19: 'n07747607_10345',
 20: 'n07747607_10358',
 21: 'n07747607_10379',
 22: 'n07747607_10420',
 23: 'n07747607_10425',
 24: 'n07747607_10436',
 25: 'n07747607_10465',
 26: 'n07747607_10482',
 27: 'n07747607_10485',
 28: 'n07747607_10491',
 29: 'n07747607_10545'}

In [11]:
images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])

In [12]:
# Load image
# ext may be different.
optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True, ext=".png")
valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
optimized_data_zscore = zscore(optimized_data, mean, std)
images_zscore = zscore(images_cpu, mean, std)

Load from ../../../saved/pack/7013/149999/n07747607_10061.png
Load from ../../../saved/pack/7013/149999/n07747607_10068.png
Load from ../../../saved/pack/7013/149999/n07747607_10110.png
Load from ../../../saved/pack/7013/149999/n07747607_10134.png
Load from ../../../saved/pack/7013/149999/n07747607_10162.png
Load from ../../../saved/pack/7013/149999/n07747607_10166.png
Load from ../../../saved/pack/7013/149999/n07747607_10178.png
Load from ../../../saved/pack/7013/149999/n07747607_10179.png
Load from ../../../saved/pack/7013/149999/n07747607_10184.png
Load from ../../../saved/pack/7013/149999/n07747607_10192.png
Load from ../../../saved/pack/7013/149999/n07747607_10204.png
Load from ../../../saved/pack/7013/149999/n07747607_10205.png
Load from ../../../saved/pack/7013/149999/n07747607_1021.png
Load from ../../../saved/pack/7013/149999/n07747607_10213.png
Load from ../../../saved/pack/7013/149999/n07747607_10217.png
Load from ../../../saved/pack/7013/149999/n07747607_10232.png
Load from

In [13]:
# Move to device
opt_image = torch.from_numpy(optimized_data_zscore).to(device)
original_image = torch.from_numpy(images_zscore).to(device)

## Obtain feature from conv layers.

In [14]:
ori_activation_maps = net.get_activation_maps(original_image, 
                                              selected_layer=selected_layer)[0]
opt_activation_maps = net.get_activation_maps(opt_image, 
                                              selected_layer=selected_layer)[0]

images_cpu_p, optimized_data_cpu, ori_activation_maps_cpu, \
    opt_activation_maps_cpu, optimized_data_cpu_scale, \
    diff_activation_maps_cpu = \
    preprocess_arrays(images_cpu, optimized_data,
                      ori_activation_maps, opt_activation_maps,
                      selected_filter, color_map=color_map)
concated_imgs = concat_imgs(images_cpu_p, optimized_data_cpu,
                            ori_activation_maps_cpu, opt_activation_maps_cpu,
                            optimized_data_cpu_scale, diff_activation_maps_cpu)

=> Register fhook 25
=> Register fhook 25


## Visualization Here.

In [16]:
save_path = os.path.join("../../../saved/pack/", exp, 
                         "concat.png")
concated_imgs.save(save_path)

In [ ]:
def main()